# Basic Risk Simulation for Ergodicity Paper - Parallel Processing

Future enhancements:
- Consider trends and discounting

In [1]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import sys
from itertools import product
from typing import List, Dict, Any
from pathlib import Path
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

from ergodic_insurance.config import ManufacturerConfig
from ergodic_insurance.manufacturer import WidgetManufacturer
from ergodic_insurance.exposure_base import RevenueExposure
from ergodic_insurance.insurance import InsurancePolicy, InsuranceLayer
from ergodic_insurance.simulation import Simulation, SimulationResults
from ergodic_insurance.ergodic_analyzer import ErgodicAnalyzer
from ergodic_insurance.loss_distributions import ManufacturingLossGenerator
from ergodic_insurance.insurance_program import EnhancedInsuranceLayer, InsuranceProgram

In [ ]:
# initial_assets_list = [5_000_000, 10_000_000, 25_000_000, 50_000_000, 100_000_000]
# atr_list = [0.5, 0.75, 1.0, 1.25, 1.5]
# ebitabl_list = [0.1, 0.125, 0.15, 0.175, 0.2]
# deductible_list = [0, 50_000, 100_000]
initial_assets_list = [5_000_000, 10_000_000, 25_000_000]
atr_list = [0.8, 0.9, 1.0, 1.1, 1.2]
ebitabl_list = [0.1, 0.125, 0.15]
deductible_list = [0, 50_000, 100_000, 250_000, 500_000]
loss_ratio_list = [0.6, 0.7, 0.8]

In [3]:
sorted_ebitabl = sorted([x * y for x, y in product(atr_list, ebitabl_list)])
print(sorted_ebitabl)
print(len(sorted_ebitabl))

[0.08000000000000002, 0.1, 0.1, 0.12, 0.12, 0.125, 0.15, 0.15, 0.18]
9


In [ ]:
NUM_SIMULATIONS = 1000
SIM_YEARS = 50
PRICING_SIMULATIONS = 50_000  # For premium estimation

from tqdm.notebook import tqdm  # progress bar

total_combos = (
    len(initial_assets_list)
    * len(atr_list)
    * len(ebitabl_list)
    * len(deductible_list)
    * len(loss_ratio_list)
)

for index, (ia, atr, ebitabl, ded, lr) in tqdm(
    enumerate(product(
        initial_assets_list,
        atr_list,
        ebitabl_list,
        deductible_list,
        loss_ratio_list
    )),
    total=total_combos,
    desc="Parameter sets",
    leave=True
):
    print(f"\nRunning simulation for Initial Assets: ${ia:,.0f}, ATR: {atr}, EBITABL: {ebitabl:.3f}, Deductible: ${ded:,.0f}, Loss Ratio: {lr:.2f}")
    INITIAL_ASSETS = ia
    ASSET_TURNOVER_RATIO = atr  # Revenue = Assets Ã— Turnover
    EBITABL = ebitabl  # EBITA after claims and insurance
    DEDUCTIBLE = ded
    LOSS_RATIO = lr

    # Set random seed for reproducibility
    base_seed = 42 + index * 1000

    ### Define the Corporate Profile #####

    # Create manufacturer configuration
    manufacturer_config = ManufacturerConfig(
        initial_assets=INITIAL_ASSETS,
        asset_turnover_ratio=ASSET_TURNOVER_RATIO,  # Revenue = Assets Ã— Turnover
        base_operating_margin=EBITABL,  # EBITA before claims and insurance (need to calibrate)
        tax_rate=0.25,  # Current US Tax Rate
        retention_ratio=0.70,  # 30% dividends
        ppe_ratio=0.00,  # 0% of assets in PPE, so there is no depreciation expense
    )

    # Create widget manufacturer
    base_manufacturer = WidgetManufacturer(manufacturer_config)

    # Create exposure base based on revenue
    exposure = RevenueExposure(state_provider=base_manufacturer)

    ## Define Losses

    cur_revenue = base_manufacturer.total_assets * base_manufacturer.asset_turnover_ratio

    generator_pricing = ManufacturingLossGenerator(
        attritional_params={
            'base_frequency': 2.85 * cur_revenue / 10_000_000,  # Scale frequency with revenue
            'severity_mean': 40_000,
            'severity_cv': 0.8,
            'revenue_scaling_exponent': 1.0,
            'reference_revenue': cur_revenue
        },
        large_params={
            'base_frequency': 0.20 * cur_revenue / 10_000_000,  # Scale frequency with revenue
            'severity_mean': 500_000,
            'severity_cv': 1.5,
            'revenue_scaling_exponent': 1.0,
            'reference_revenue': cur_revenue
        },
        catastrophic_params={
            'base_frequency': 0.02 * cur_revenue / 10_000_000,  # Scale frequency with revenue
            'severity_xm': 5_000_000,
            'severity_alpha': 2.5,
            'revenue_scaling_exponent': 1.0,
            'reference_revenue': cur_revenue
        },
        seed=base_seed
    )


    deductible = DEDUCTIBLE
    policy_limit = 100_000_000_000  # No upper limit for pricing purposes

    ### Run Pricing Simulation #####
    # Assume the insurer has perfect knowledge of the loss distribution

    pricing_simulation_years = PRICING_SIMULATIONS

    total_insured_loss = 0.0
    insured_loss_list = []

    total_retained_loss = 0.0
    retained_loss_list = []

    for yr in range(pricing_simulation_years):
        loss_events, loss_meta = generator_pricing.generate_losses(duration=1, revenue=base_manufacturer.base_revenue)
        for loss_event in loss_events:
            insured_loss = max(min(loss_event.amount - deductible, policy_limit),0)
            
            total_insured_loss += insured_loss
            insured_loss_list.append(insured_loss)

            retained_loss = loss_event.amount - insured_loss
            total_retained_loss += retained_loss
            retained_loss_list.append(retained_loss)

    average_annual_insured_loss = total_insured_loss / pricing_simulation_years
    average_annual_retained_loss = total_retained_loss / pricing_simulation_years
    print(f"Average Annual Insured Loss: ${average_annual_insured_loss:,.0f}")
    print(f"Average Annual Retained Loss: ${average_annual_retained_loss:,.0f}")


    ground_up_losses = np.asarray(insured_loss_list, dtype=float) + np.asarray(retained_loss_list, dtype=float)
    EXCESS_KURTOSIS = pd.Series(ground_up_losses).kurtosis()
    print(f"Ground-Up Excess Kurtosis: {EXCESS_KURTOSIS:.2f}")

    loss_ratio = LOSS_RATIO

    annual_premium = average_annual_insured_loss / loss_ratio
    print(f"Annual Premium: ${annual_premium:,.0f}")

    total_cost_of_risk = annual_premium + average_annual_retained_loss
    print(f"Total Annual Cost of Risk: ${total_cost_of_risk:,.0f}")


    cur_operating_income = base_manufacturer.calculate_operating_income(cur_revenue)

    cur_net_income = base_manufacturer.calculate_net_income(
                        operating_income=cur_operating_income,
                        collateral_costs=0.0,
                        use_accrual=True,
                        time_resolution="annual",
                    )

    # target_net_income = base_manufacturer.base_revenue * target_ebita_margin * (1 - base_manufacturer.tax_rate)
    # target_net_income


    # net_margin_diff = abs(cur_net_income - target_net_income) / cur_revenue

    # assert net_margin_diff < 0.0005, f"Net income not within 0.05% of target ({net_margin_diff:.2%} difference)"


    net_margin = cur_net_income / cur_revenue
    print(f"Net Margin after insurance: {net_margin:.2%}")
    print(f"EBITA Margin after insurance: {net_margin / (1 - base_manufacturer.tax_rate):.2%}")

    ## Define the Insurance Program

    all_layers = EnhancedInsuranceLayer(
                    attachment_point=deductible,
                    limit=policy_limit,
                    limit_type='per-occurrence',
                    base_premium_rate=annual_premium / policy_limit
                )

    program = InsuranceProgram([all_layers])

    # total_premium = program.calculate_annual_premium()
    
    ### Set Up the Simulation #######

    from ergodic_insurance.monte_carlo import MonteCarloEngine, MonteCarloResults, MonteCarloConfig

    def setup_simulation_engine(n_simulations=10_000, n_years=10, parallel=False, insurance_program=None, seed=base_seed + 100):
        """Set up Monte Carlo simulation engine."""
        generator = ManufacturingLossGenerator(
            attritional_params={
                'base_frequency': 2.85 * cur_revenue / 10_000_000,  # Scale frequency with revenue
                'severity_mean': 40_000,
                'severity_cv': 0.8,
                'revenue_scaling_exponent': 1.0,
                'reference_revenue': cur_revenue
            },
            large_params={
                'base_frequency': 0.20 * cur_revenue / 10_000_000,  # Scale frequency with revenue
                'severity_mean': 500_000,
                'severity_cv': 1.5,
                'revenue_scaling_exponent': 1.0,
                'reference_revenue': cur_revenue
            },
            catastrophic_params={
                'base_frequency': 0.02 * cur_revenue / 10_000_000,  # Scale frequency with revenue
                'severity_xm': 5_000_000,
                'severity_alpha': 2.5,
                'revenue_scaling_exponent': 1.0,
                'reference_revenue': cur_revenue
            },
            seed=seed
        )

        # Create simulation config
        config = MonteCarloConfig(
            n_simulations=n_simulations,
            n_years=n_years,
            n_chains=4,
            parallel=parallel,
            n_workers=None,
            chunk_size=max(1000, n_simulations // 10),
            use_float32=True,
            cache_results=False,
            progress_bar=True,
            ruin_evaluation=[5, 10, 15, 20, 25, 30, 35, 40, 45, 50],
            seed=seed + 450
        )
        
        if insurance_program is None:
            insurance_program = InsuranceProgram(
                                    layers=[],  # Empty list to define no coverage
                                    deductible=0.0,  # No deductible needed since all losses are retained
                                    pricer=None,
                                    name="No Insurance"
                                )

        # Create engine
        engine = MonteCarloEngine(
            loss_generator=generator,
            insurance_program=insurance_program,
            manufacturer=base_manufacturer,
            config=config
        )
        
        return engine

    # Create engine
    print("Setting up Monte Carlo engine with Insurance...")
    engine = setup_simulation_engine(n_simulations=NUM_SIMULATIONS,
                                    n_years=SIM_YEARS,
                                    parallel=False,
                                    insurance_program=program,
                                    seed=base_seed + 100)
    print(f"Engine configured: {engine.config.n_simulations:,} simulations, {engine.config.n_years} years")
    print(f"Parallel processing: {engine.config.parallel}")
    print(f"Number of chains: {engine.config.n_chains}")
    print(f"Deductible: {engine.insurance_program.deductible}")

    ### Set Up the Simulation Without Insurance

    # Create engine without insurance
    print("Setting up Monte Carlo engine without Insurance...")
    engine_no_ins = setup_simulation_engine(n_simulations=NUM_SIMULATIONS,
                                            n_years=SIM_YEARS,
                                            parallel=False,
                                            insurance_program=None,
                                            seed=base_seed + 200)
    print(f"Engine configured: {engine_no_ins.config.n_simulations:,} simulations, {engine_no_ins.config.n_years} years")
    print(f"Parallel processing: {engine_no_ins.config.parallel}")
    print(f"Number of chains: {engine_no_ins.config.n_chains}")
    print(f"Deductible: {engine_no_ins.insurance_program.deductible}")

    ## Run the Simulation

    filename = f"results\Cap ({INITIAL_ASSETS/1_000_000:.0f}M) -\
    ATR ({ASSET_TURNOVER_RATIO}) -\
    EBITABL ({EBITABL}) -\
    XS_Kurt ({EXCESS_KURTOSIS:.0f}) -\
    Ded ({DEDUCTIBLE/1_000:.0f}K) -\
    LR ({LOSS_RATIO}) -\
    {NUM_SIMULATIONS/1_000:.0f}K Sims -\
    {SIM_YEARS} Yrs.pkl"

    filename_no_ins = f"results\Cap ({INITIAL_ASSETS/1_000_000:.0f}M) -\
    ATR ({ASSET_TURNOVER_RATIO}) -\
    EBITABL ({EBITABL}) -\
    XS_Kurt ({EXCESS_KURTOSIS:.0f}) -\
    NOINS -\
    {NUM_SIMULATIONS/1_000:.0f}K Sims -\
    {SIM_YEARS} Yrs.pkl"

    results = engine.run()

    import pickle

    with open(filename, "wb") as f:
        pickle.dump(results, f, protocol=pickle.HIGHEST_PROTOCOL)

    file_exists_no_ins = Path(filename_no_ins).exists()

    if file_exists_no_ins:
        print(f"Skipping no-insurance simulation run, already exists: {filename_no_ins}")
    else:
        results_no_ins = engine_no_ins.run()

        with open(filename_no_ins, "wb") as f:
            pickle.dump(results_no_ins, f, protocol=pickle.HIGHEST_PROTOCOL)

    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    from ergodic_insurance.visualization.core import WSJ_COLORS
    import numpy as np

    # Print results
    print(f"\nRuin Probabilities by Time Horizon:")
    for yr, prob in results.ruin_probability.items():
        print(f"  {yr} years: {prob*100:.2f}%")

    print(f"\nConvergence achieved: {'Yes' if results.convergence else 'No'}")

# Load Pickled Results

In [5]:
# # Import required libraries
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# from typing import List, Dict, Any
# import warnings
# import sys
# from pathlib import Path
# warnings.filterwarnings('ignore')

# # Set plotting style
# plt.style.use('seaborn-v0_8-darkgrid')
# sns.set_palette("husl")

# from ergodic_insurance.config import ManufacturerConfig
# from ergodic_insurance.manufacturer import WidgetManufacturer
# from ergodic_insurance.exposure_base import RevenueExposure
# from ergodic_insurance.insurance import InsurancePolicy, InsuranceLayer
# from ergodic_insurance.simulation import Simulation, SimulationResults
# from ergodic_insurance.ergodic_analyzer import ErgodicAnalyzer
# from ergodic_insurance.loss_distributions import ManufacturingLossGenerator
# from ergodic_insurance.insurance_program import EnhancedInsuranceLayer, InsuranceProgram

# # Read the specified pickle file (tries results/ directory first)
# results_filename = "Cap (10M) - ATR (1.0) - EBITA (0.05) - XS_Kurt (3728) - Ded (50K) - LR (0.7).pkl"

# candidate_paths = [Path("results") / results_filename, Path(results_filename)]

# if results is None:
#     for p in candidate_paths:
#         if p.exists():
#             with open(p, "rb") as f:
#                 results = pickle.load(f)
#             print(f"Loaded pickle from: {p}")
#             break

#     if results is None:
#         print(f"File not found in: {[str(p) for p in candidate_paths]}")
#     else:
#         # If this is a SimulationResults object, show a brief summary
#         if hasattr(results, "summary"):
#             try:
#                 print(results.summary())
#             except Exception as e:
#                 print(f"Could not print summary: {e}")
#         else:
#             print(f"Loaded object type: {type(results)}")
# else:
#     print("Results with insurance already loaded.")

# # Read the specified pickle file (tries results/ directory first)
# results_no_ins_filename = "Cap (10M) - ATR (1.0) - EBITA (0.05) - XS_Kurt (3728) - NOINS.pkl"

# candidate_paths = [Path("results") / results_no_ins_filename, Path(results_no_ins_filename)]

# if results_no_ins is None:
#     for p in candidate_paths:
#         if p.exists():
#             with open(p, "rb") as f:
#                 results_no_ins = pickle.load(f)
#             print(f"Loaded pickle from: {p}")
#             break

#     if results_no_ins is None:
#         print(f"File not found in: {[str(p) for p in candidate_paths]}")
#     else:
#         # If this is a SimulationResults object, show a brief summary
#         if hasattr(results_no_ins, "summary"):
#             try:
#                 print(results_no_ins.summary())
#             except Exception as e:
#                 print(f"Could not print summary: {e}")
#         else:
#             print(f"Loaded object type: {type(results_no_ins)}")
# else:
#     print("Results without insurance already loaded.")
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots
# from ergodic_insurance.visualization.core import WSJ_COLORS
# import numpy as np

# # Print results
# print(f"\nRuin Probabilities by Time Horizon:")
# for yr, prob in results.ruin_probability.items():
#     print(f"  {yr} years: {prob*100:.2f}%")

# print(f"\nConvergence achieved: {'Yes' if results.convergence else 'No'}")

# # --------------------------------------
# # Time-Average Growth Rate Histogram (1,1)
# # --------------------------------------
# fig = go.Figure()

# # Growth rates with insurance
# growth_rates = np.asarray(results.growth_rates, dtype=float)
# growth_rates = growth_rates[np.isfinite(growth_rates)]

# # Growth rates without insurance
# growth_rates_no_ins = np.asarray(results_no_ins.growth_rates, dtype=float)
# growth_rates_no_ins = growth_rates_no_ins[np.isfinite(growth_rates_no_ins)]

# growth_pct = growth_rates * 100.0
# mean_growth = np.nanmean(growth_pct)
# median_growth = np.nanmedian(growth_pct)
# p5, p25, p95 = np.nanpercentile(growth_pct, [5, 25, 95])
# print(f"Growth Rate Summary With Insurance (%): Mean={mean_growth:.2f}, Median={median_growth:.2f}, 25th={p25:.2f}, 5th={p5:.2f}")

# growth_pct_no_ins = growth_rates_no_ins * 100.0
# mean_growth_no_ins = np.nanmean(growth_pct_no_ins)
# median_growth_no_ins = np.nanmedian(growth_pct_no_ins)
# p5_no_ins, p25_no_ins, p95_no_ins = np.nanpercentile(growth_pct_no_ins, [5, 25, 95])
# print(f"Growth Rate Summary Without Insurance (%): Mean={mean_growth_no_ins:.2f}, Median={median_growth_no_ins:.2f}, 25th={p25_no_ins:.2f}, 5th={p5_no_ins:.2f}")

# # Plot Growth Rate With Insurance
# # Kernel Density Estimate (Gaussian) instead of histogram
# xs = np.linspace(growth_pct.min(), growth_pct.max(), 400)
# n = growth_pct.size
# std = np.std(growth_pct, ddof=1)
# if std == 0 or n < 2:
#     # Fallback: flat line at 0
#     density = np.zeros_like(xs)
# else:
#     bw = 1.06 * std * n ** (-1/5)  # Silverman's rule
#     inv_norm = 1.0 / (bw * np.sqrt(2 * np.pi))
#     diffs = (xs[:, None] - growth_pct[None, :]) / bw
#     density = inv_norm * np.exp(-0.5 * diffs**2).mean(axis=1)

# fig.add_trace(
#     go.Scatter(
#         x=xs,
#         y=density,
#         mode='lines',
#         name='Insurance',
#         line=dict(color=WSJ_COLORS.get('blue', '#1f77b4'), width=2)
#     )
# )

# # Plot Growth Rate Without Insurance
# # Kernel Density Estimate (Gaussian) instead of histogram
# xs_no_ins = np.linspace(growth_pct_no_ins.min(), growth_pct_no_ins.max(), 400)
# n_no_ins = growth_pct_no_ins.size
# std_no_ins = np.std(growth_pct_no_ins, ddof=1)
# if std_no_ins == 0 or n_no_ins < 2:
#     # Fallback: flat line at 0
#     density = np.zeros_like(xs_no_ins)
# else:
#     bw = 1.06 * std_no_ins * n_no_ins ** (-1/5)  # Silverman's rule
#     inv_norm_no_ins = 1.0 / (bw * np.sqrt(2 * np.pi))
#     diffs_no_ins = (xs_no_ins[:, None] - growth_pct_no_ins[None, :]) / bw
#     density_no_ins = inv_norm_no_ins * np.exp(-0.5 * diffs_no_ins**2).mean(axis=1)

# # Set x-axis to central 99.5% of both distributions (trim 0.25% each tail)
# try:
#     lower_q, upper_q = .5, 99.5
#     lo_ins, hi_ins = np.nanpercentile(growth_pct, [lower_q, upper_q])
#     lo_no_ins, hi_no_ins = np.nanpercentile(growth_pct_no_ins, [lower_q, upper_q])
#     x_min = min(lo_ins, lo_no_ins)
#     x_max = max(hi_ins, hi_no_ins)
#     pad = 0.01 * (x_max - x_min) if x_max > x_min else 0.0
#     fig.update_xaxes(range=[x_min - pad, x_max + pad])
# except Exception as e:
#     print(f"Could not apply 99.5% central x-range: {e}")

# # Adjust y-axis label to reflect density
# fig.update_yaxes(title_text="Density")

# # Recompute KDEs on clipped range [-10, 5] and update plot
# lower, upper = -5.0, 5.0
# # Clear existing traces, shapes, annotations
# fig.data = ()
# fig.layout.shapes = ()
# fig.layout.annotations = ()

# # Clip datasets
# ins_clip = growth_pct[(growth_pct >= lower) & (growth_pct <= upper)]
# no_ins_clip = growth_pct_no_ins[(growth_pct_no_ins >= lower) & (growth_pct_no_ins <= upper)]

# xs_clip = np.linspace(lower, upper, 400)

# def kde(arr, xs):
#     n = arr.size
#     if n < 2:
#         return np.zeros_like(xs)
#     std_ = np.std(arr, ddof=1)
#     if std_ == 0:
#         return np.zeros_like(xs)
#     bw = 1.06 * std_ * n ** (-1/5)
#     inv = 1.0 / (bw * np.sqrt(2 * np.pi))
#     diffs = (xs[:, None] - arr[None, :]) / bw
#     return inv * np.exp(-0.5 * diffs**2).mean(axis=1)

# dens_ins = kde(ins_clip, xs_clip) if ins_clip.size else np.zeros_like(xs_clip)
# dens_no  = kde(no_ins_clip, xs_clip) if no_ins_clip.size else np.zeros_like(xs_clip)

# # Add traces
# fig.add_trace(go.Scatter(
#     x=xs_clip, y=dens_ins, mode='lines', name='Insurance (clipped)',
#     line=dict(color=WSJ_COLORS.get('blue', '#1f77b4'), width=2)
# ))
# fig.add_trace(go.Scatter(
#     x=xs_clip, y=dens_no, mode='lines', name='No Insurance (clipped)',
#     line=dict(color=WSJ_COLORS.get('orange', '#ff7f0e'), width=2)
# ))

# # Stats on clipped data
# if ins_clip.size:
#     mean_ins = np.nanmean(ins_clip)
#     p25_ins = np.nanpercentile(ins_clip, 25)
#     # Add vertical lines for mean and median
#     fig.add_vline(x=mean_ins, 
#                 line_dash='dash', 
#                 line_color=WSJ_COLORS.get('blue', '#1f77b4'), 
#                 annotation_text='Mean (Ins)', 
#                 annotation_position='top right')
#     fig.add_vline(x=p25_ins, 
#                 line_dash='dot', 
#                 line_color=WSJ_COLORS.get('red', '#d62728'), 
#                 annotation_text='25th pct (Ins)', 
#                 annotation_position='top left')
# if no_ins_clip.size:
#     mean_no = np.nanmean(no_ins_clip)
#     p25_no = np.nanpercentile(no_ins_clip, 25)
#     # Add vertical lines for mean and median
#     fig.add_vline(x=mean_no, 
#                 line_dash='dash', 
#                 line_color=WSJ_COLORS.get('orange', '#ff7f0e'), 
#                 annotation_text='Mean (No Ins)', 
#                 annotation_position='bottom right')
#     fig.add_vline(x=p25_no, 
#                 line_dash='dot', 
#                 line_color=WSJ_COLORS.get('red', '#d62728'),
#                 annotation_text='25th pct (No Ins)', 
#                 annotation_position='bottom left')


# # Update layout
# fig.update_layout(
#     title={
#         "text": "Time-Average Growth Rate Histogram",
#         "x": 0.5,
#         "xanchor": "center"
#     },
#     template='plotly_white',
#     showlegend=True,
#     legend=dict(
#         orientation='h',
#         x=0.5,
#         xanchor='center',
#         y=-0.2,
#         yanchor='top'
#     ),
#     width=800,
#     height=500
# )

# # Axes titles
# fig.update_xaxes(title_text="Time-Average Growth Rate (%)",
#                 range=[lower, upper])

# fig.show()
# import plotly.io as pio

# fig_filename = f"results\Cap ({INITIAL_ASSETS/1_000_000:.0f}M) -\
#  ATR ({ASSET_TURNOVER_RATIO}) -\
#  EBITABL ({EBITABL}) -\
#  XS_Kurt (high) -\
#  Ded ({DEDUCTIBLE/1_000:.0f}K) -\
#  LR ({LOSS_RATIO}) -\
#  {NUM_SIMULATIONS/1_000:.0f}K Sims -\
#  {SIM_YEARS} Yrs - Growth Rate.png"

# pio.write_image(fig, fig_filename, scale=2) # Adjust scale as needed
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots
# from ergodic_insurance.visualization.core import WSJ_COLORS
# import numpy as np

# # Print results
# print(f"\nRuin Probabilities by Time Horizon:")
# for yr, prob in results.ruin_probability.items():
#     print(f"  {yr} years: {prob*100:.2f}%")

# print(f"\nConvergence achieved: {'Yes' if results.convergence else 'No'}")

# # --------------------------------------
# # Ruin probability comparison (1,2)
# # --------------------------------------
# # Single figure without subplots
# fig = go.Figure()
# fig.add_trace(
#     go.Scatter(
#         x=list(results.ruin_probability.keys()),
#         y=[prob * 100 for prob in results.ruin_probability.values()],
#         mode='lines+markers',
#         name='Insurance',
#         line=dict(width=2, color=WSJ_COLORS.get('blue', '#1f77b4')),
#         marker=dict(size=8)
#     )
# )
# fig.add_trace(
#     go.Scatter(
#         x=list(results_no_ins.ruin_probability.keys()),
#         y=[prob * 100 for prob in results_no_ins.ruin_probability.values()],
#         mode='lines+markers',
#         name='No Insurance',
#         line=dict(width=2, color=WSJ_COLORS.get('orange', '#ff7f0e')),
#         marker=dict(size=8)
#     )
# )

# # Update layout
# fig.update_layout(
#     title={
#         "text": "Ruin Probability by Time Horizon",
#         "x": 0.5,
#         "xanchor": "center"
#     },
#     template='plotly_white',
#     showlegend=True,
#     legend=dict(
#         orientation='h',
#         x=0.5,
#         xanchor='center',
#         y=-0.2,
#         yanchor='top'
#     ),
#     width=800,
#     height=500
# )

# # Axes titles
# fig.update_xaxes(title_text="Time Horizon (years)")
# fig.update_yaxes(title_text="Ruin Probability (%)")

# fig.show()
# fig_filename = f"results\Cap ({INITIAL_ASSETS/1_000_000:.0f}M) -\
#  ATR ({ASSET_TURNOVER_RATIO}) -\
#  EBITABL ({EBITABL}) -\
#  XS_Kurt (high) -\
#  Ded ({DEDUCTIBLE/1_000:.0f}K) -\
#  LR ({LOSS_RATIO}) -\
#  {NUM_SIMULATIONS/1_000:.0f}K Sims -\
#  {SIM_YEARS} Yrs - RoR.png"

# pio.write_image(fig, fig_filename, scale=2) # Adjust scale as needed